In [1]:
System.out.println("Hello World")

Hello World


In [2]:
%maven ai.djl:api:0.31.0
%maven ai.djl.onnxruntime:onnxruntime-engine:0.31.0
%maven org.slf4j:slf4j-simple:1.7.36

In [36]:
import ai.djl.inference.*;
import ai.djl.modality.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.types.*;
import ai.djl.repository.zoo.*;
import ai.djl.translate.*;
import java.util.*;
import java.nio.file.Path;
import java.nio.file.Paths;

In [37]:
public static class IrisFlower {

    public float sepalLength;
    public float sepalWidth;
    public float petalLength;
    public float petalWidth;

    public IrisFlower(float sepalLength, float sepalWidth, float petalLength, float petalWidth) {
        this.sepalLength = sepalLength;
        this.sepalWidth = sepalWidth;
        this.petalLength = petalLength;
        this.petalWidth = petalWidth;
    }
}

In [48]:
 
public static class MyTranslator implements NoBatchifyTranslator<IrisFlower, Classifications> {

    private final List<String> synset;

    public MyTranslator() {
        // species name
        synset = Arrays.asList("setosa", "versicolor", "virginica");
    }

    @Override
    public NDList processInput(TranslatorContext ctx, IrisFlower input) {
        float[] data = {(float)input.sepalLength, (float)input.sepalWidth, (float)input.petalLength, (float)input.petalWidth};
        NDArray array = ctx.getNDManager().create(data, new Shape(1, 4));
        array = array.toType(DataType.FLOAT32, false); // Ensure it's float32
        array.setName("float_input"); // ✅ Set input name to match ONNX model
        return new NDList(array);
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        float[] data = list.get(0).toFloatArray();
        List<Double> probabilities = new ArrayList<>(data.length);
        for (float f : data) {
            probabilities.add((double) f);
        }
        return new Classifications(synset, probabilities);
    }
}

In [46]:

Path modelPath = Paths.get("logreg_iris.onnx");
Criteria<IrisFlower, Classifications> criteria = Criteria.builder()
        .setTypes(IrisFlower.class, Classifications.class)
        .optModelPath(modelPath)
        .optTranslator(new MyTranslator())
        .optEngine("OnnxRuntime") // use OnnxRuntime engine by default
        .build();
ZooModel<IrisFlower, Classifications> model = criteria.loadModel();

In [ ]:
Predictor<IrisFlower, Classifications> predictor = model.newPredictor();
IrisFlower info = new IrisFlower(1.0f, 2.0f, 3.0f, 4.0f);
NDList inputNDList = translator.processInput(new TranslatorContext(null), info);
System.out.println("NDArray DataType: " + inputNDList.get(0).getDataType());
Classifications result = predictor.predict(info);
System.out.println(result);

EvalException: java.lang.IllegalStateException: DataType mismatch, Required float, Actual int64